In [129]:
import tkinter
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from tkinter import filedialog

In [130]:
def get_steam_library():
    tkinter.Tk().withdraw() # prevents an empty tkinter window from appearing
    folder_path = filedialog.askdirectory()
    return folder_path

In [4]:
os.getcwd() # 'F:\\Bibliothek\\Python_Projekte'
path = get_steam_library()
unit_filepath = path + '\\steamapps\\common\\Warhammer 40000 Gladius - Relics of War\\Data\\World\\Units\\ChaosSpaceMarines'
icon_filepath = path + '\\steamapps\\common\\Warhammer 40000 Gladius - Relics of War\\Data\\Video\\Textures\\Icons\Attributes'
# unit_filepath = 'S:\\SteamLibrary\\steamapps\\common\\Warhammer 40000 Gladius - Relics of War\\Data\\World\\Units\\ChaosSpaceMarines'
# icon_filepath = 'S:\\SteamLibrary\\steamapps\\common\\Warhammer 40000 Gladius - Relics of War\\Data\\Video\\Textures\\Icons\Attributes'
os.chdir(unit_filepath)
os.getcwd()

'S:\\SteamLibrary\\steamapps\\common\\Warhammer 40000 Gladius - Relics of War\\Data\\World\\Units\\ChaosSpaceMarines'

In [6]:
os.listdir()

['ChaosCultist.xml',
 'ChaosLandRaider.xml',
 'ChaosLord.xml',
 'ChaosRhino.xml',
 'ChaosSpaceMarine.xml',
 'ChaosSpawn.xml',
 'DaemonPrince.xml',
 'DarkDisciple.xml',
 'Defiler.xml',
 'Forgefiend.xml',
 'Havoc.xml',
 'Headquarters.xml',
 'Helbrute.xml',
 'Heldrake.xml',
 'KhorneBerzerker.xml',
 'MasterOfPossession.xml',
 'Maulerfiend.xml',
 'NoctilithCrown.xml',
 'Obliterator.xml',
 'Venomcrawler.xml',
 'Warpsmith.xml',
 'WarpTalon.xml']

In [106]:
def create_soups():
    pot = []
    files = os.listdir()
    for file in files:
        with open(file, 'r') as f:
            data = f.read()
            soup = BeautifulSoup(data, 'xml')
            pot.append(soup)
    return pot
pot = create_soups()

In [171]:
unit_stats = ['name','armor', 'hitpointsMax', 'moraleMax', 'movementMax', 'itemSlots', 'cargoSlots','size','rowSize']
unit_traits = []
unit_costs = ['productionCost','foodCost', 'oreCost', 'energyCost', 'influcenceCost',
              'foodUpkeep', 'oreUpkeep', 'energyUpkeep', 'influenceUpkeep']

unit_attributes = unit_stats + unit_costs

def edit_list(data):
    for index, elem in enumerate(data):
        data[index] = re.sub(r"(\w)([A-Z])", r"\1 \2", elem).title().replace(' Max','')
    return data

In [162]:
def get_unit_stats(soup):
    stats = {}
    name = soup.find_all('unit')[1]
    group = soup.find('group')
    effect = soup.find('effects')
    keys = edit_list(unit_attributes)
    for index, s in enumerate(unit_stats + unit_costs):
        try:
            if s=='name':
                stats[keys[index]] = re.sub(r"(\w)([A-Z])", r"\1 \2", name.attrs['mesh'].split('/')[-1])
            elif s=='size': 
                stats[keys[index]] = group.attrs['size']
            elif s=='rowSize': 
                stats[keys[index]] = group.attrs['rowSize']
            else:
                stat = effect.find(s)
                stats[keys[index]] = stat.attrs['base']
        except:
            stats[keys[index]] = None;    
    return stats         

def print_unit_stats(unit):
    for key, value in unit.items():    
        print("{:<15} {:<10} ".format(key, value))   

In [163]:
get_unit_stats(pot[2])

{'Name': 'Chaos Lord',
 'Armor': '8',
 'Hitpoints': '9.0',
 'Morale': '12',
 'Movement': '3',
 'Item Slots': '6',
 'Cargo Slots': None,
 'Size': None,
 'Row Size': None,
 'Production Cost': '42.0',
 'Food Cost': '30.0',
 'Ore Cost': '30.0',
 'Energy Cost': None,
 'Influcence Cost': None,
 'Food Upkeep': '3.0',
 'Ore Upkeep': None,
 'Energy Upkeep': None,
 'Influence Upkeep': '6.0'}

In [9]:
def get_unit_traits(soup):
    traits = []
    soup = soup.find_all('trait')
    for trait in soup:
        try: # catching a KeyError. Not every trait has the attribute requiredUpgrade
            name = trait.attrs['name'].split('/')[-1]
            if (trait.attrs['requiredUpgrade']): name = "Upgrade-" + name
            traits.append(name)
        except:
            traits.append(name)
    return traits

In [10]:
get_unit_traits(pot[3])

['Upgrade-ExtraVehicleArmour',
 'Upgrade-SoulBlaze',
 'Tank',
 'Transport',
 'Vehicle']

In [11]:
def get_action_names(soup):
    action_names = [] # names
    actions = {} # name - details
    action_soup = [value for value in soup.find('actions').contents if value != '\n']
    for a in action_soup:
        value = a.name
        if a.name == 'throwGrenade':
            value = a.attrs['weaponSlotName']
        if value == 'genericUnitAbility':
            value = a.attrs['name'].split('/')[1]
        action_names.append(value)
    return action_names

In [12]:
get_action_names(pot[1])

['attack', 'die', 'move', 'DirgeCaster', 'createSmokeScreen']

In [13]:
def get_weapon_names(soup):
    attributes = []
    for entry in soup.find_all(name='weapon'):
        try:
            name = None
            name = entry.attrs['name']
            if entry.attrs['requiredUpgrade']: attributes.append("Upgrade-" + entry.attrs['name'])
        except:
            if name: attributes.append(name)
            else: continue
    return attributes

def get_weapon_id(weapon_df, weapon_list):
    for weapon in weapon_list:
        weapon_df['weapon_list']

In [14]:
get_weapon_names(pot[1])

['TwinLinkedLascannon',
 'TwinLinkedHeavyBolter',
 'Upgrade-HavocLauncher',
 'DestroyerBlades']

In [22]:
def extract_data(soup):
    stats = get_unit_stats(soup) # dict
    weapons = get_weapon_names(soup) # list
    actions = get_action_names(soup) # list
    stats['weapons'] = weapons
    stats['actions'] = actions
    return stats

In [144]:
def edit_dict(data):
    for key in data.keys():
        key = re.sub(r"(\w)([A-Z])", r"\1 \2", key).title()
    return data
#edit_names(unit_attributes)

In [168]:
cultist = get_unit_stats(pot[1])
c = {'name':"A",'test':0}
pd.DataFrame(data=[[0,0],[0,0]],columns=["a","b"])
pd.DataFrame(cultist, unit_attributes)
pd.DataFrame(edit_dict(cultist),[0])

,Name,Armor,Hitpoints,Morale,Movement,Item Slots,Cargo Slots,Size,Row Size,Production Cost,Food Cost,Ore Cost,Energy Cost,Influcence Cost,Food Upkeep,Ore Upkeep,Energy Upkeep,Influence Upkeep
0,Chaos Land Raider,10,64.0,12,4,None,3,1,2,60.0,None,160.0,None,None,None,8.0,None,None


In [170]:
pot[3]

<?xml version="1.0" encoding="utf-8"?>
<unit dlc="Supplement3">
<model>
<unit explosionsBone="Bone001" idleAnimation="Units/ChaosSpaceMarines/ChaosRhinoIdle" idleAnimationCount="3" idleContinuously="1" material="Units/ChaosSpaceMarines/ChaosRhino" mesh="Units/ChaosSpaceMarines/ChaosRhino" scale="0.9 0.9 0.9" uvAnimated="1"/>
</model>
<group memberDeltaX="100" memberDeltaY="110" rowSize="2" size="1"/>
<weapons>
<weapon name="CombiBolter">
<model>
<projectileWeapon effectFaceWeight="0" muzzleBone=".Muzzle" muzzleCount="2"/>
</model>
</weapon>
<weapon name="HavocLauncher" requiredUpgrade="ChaosSpaceMarines/HavocLauncher">
<model>
<missileWeapon bone="Bone001" material="Units/ChaosSpaceMarines/ChaosRhino" mesh="Weapons/ChaosSpaceMarines/ChaosRhinoHavocLauncher" missileScale="0.5" muzzleBone=".Muzzle" muzzleCount="6"/>
</model>
</weapon>
<weapon name="DestroyerBlades" slotName="DestroyerBlades">
<model>
<weapon bone="Bone001" material="Units/ChaosSpaceMarines/ChaosRhino" mesh="Weapons/Chaos